In [ ]:
#import sys
#sys.path.append("D:/gnn-cloud-manufacturing/")

import torch
from torch.optim import Adam
from tqdm.auto import trange
import matplotlib.pyplot as plt
from tqdm.auto import trange
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.validation import objvalue, construct_delta, Dataset
from cloudmanufacturing.graph import os_type
from cloudmanufacturing.graphconv import GNN
import dgl
from dgl.dataloading import GraphDataLoader
import pickle
import torch.nn.functional as F
import logging
import torch
from torch.optim import Adam


In [ ]:
with open('../data/train_dataset/train_data_sheet_names.pickle' , 'rb') as s:
    sheet_names = pickle.load(s)
dataset = read_fatahi_dataset('../data/train_dataset/train_data_OPTIMAL.xlsx', sheet_names)

In [ ]:
with open('../data/train_dataset/train_data_solve.pickle' , 'rb') as f:
    # Загружаем dgl графы в список
    DGList = pickle.load(f)

In [ ]:
DGList.pop(26)
dataset.pop(26)

In [ ]:
train_dataset = Dataset(DGList[:45], dataset[:45])
val_dataset = Dataset(DGList[45:], dataset[45:])

In [ ]:
logger = logging.getLogger("../data/GNN_run")
logger.setLevel(logging.INFO)

fh = logging.FileHandler("../data/GNN_run.log")

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)

In [ ]:
n_epochs = 5000
# Списки значений параметров, которые вы хотите протестировать
lr_values = [0.001, 0.01, 0.1]
n_layers_values = [1, 2, 3]
batch_size_values = [4, 8]

results_file = open("../data/results_objvalues.txt", "a")

logger.info('Start training')

try:

    for lr in lr_values:
        for n_layers in n_layers_values:
            for batch_size in batch_size_values:
                # Создание модели с новыми параметрами
                model = GNN(ins_dim=1, ino_dim=20, out_dim=16, n_layers=n_layers)
                optim = Adam(model.parameters(), lr=lr)
                
                train_loader = GraphDataLoader(
                    train_dataset,
                    batch_size=batch_size,
                    drop_last=False,
                    shuffle=True
                )

                val_loader = GraphDataLoader(
                    val_dataset,
                    batch_size=batch_size,
                    drop_last=False,
                    shuffle=False
                )

                # Здесь должен быть код для загрузки данных и создания DataLoader'ов

                # Запуск обучения
                for epoch in trange(n_epochs):
                    train_objvalue = []
                    for batch, idx in train_loader:
                        logits = model(batch)
                        target = batch.edata['target'][os_type]
                        loss = F.binary_cross_entropy_with_logits(logits, target)
                        batch_loss = loss.item()
                        optim.zero_grad()
                        loss.backward()
                        optim.step()
                        logger.info(f"{epoch} epoch batch loss is {batch_loss}")

                        # Validation

                        # Save results to file

                        
                        for i, graph in enumerate(dgl.unbatch(batch)):
                            problem = dataset[idx[i]]
                            with graph.local_scope():
                                pred_gamma = model.predict(graph, problem)
                            train_objvalue.append(
                                objvalue(problem, pred_gamma, construct_delta(problem, pred_gamma))
                            )

                    

                    train_loss = sum(train_objvalue)/len(train_objvalue)
                    val_loss = validate(model, val_loader, dataset)

                    if epoch%10==0:
                        print('Epoch: ', epoch, 'Train: ', train_loss, 'Val: ', val_loss)  
                        print(f"batch_size: {batch_size}, n_layers: {n_layers}, lr: {lr}, epoch: {epoch}, train_loss: {train_loss}, val_loss: {val_loss}\n")

                        results_file.write(f"batch_size: {batch_size}, n_layers: {n_layers}, lr: {lr}, epoch: {epoch}, train_loss: {train_loss}, val_loss: {val_loss}\n")

    results_file.close()

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    results_file.close()
